In [ ]:
import pandas as pd

import plotly
import plotly.offline as py
import plotly.graph_objs as go
!pip install plotly_express
import plotly_express as px

from sklearn.cluster import KMeans
from sklearn.metrics import silhouette_score
import numpy.ma as ma
import matplotlib.pyplot as plt


Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
# Connect to drive
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

# Check folder files
%cd "/content/drive/Shareddrives/C23-PS139-ML"
%ls -al


Mounted at /content/drive
/content/drive/Shareddrives/C23-PS139-ML
total 5075
-rw------- 1 root root  106116 Jun 14 14:47  content_based_data.csv
drwx------ 2 root root    4096 May 27 14:59  csv/
drwx------ 2 root root    4096 May 30 12:48  data/
-rw------- 1 root root  536792 Jun 14 16:46  destination_recommender_model.h5
-rw------- 1 root root 2673790 Jun 14 14:46  dfs.csv
-rw------- 1 root root     275 May 29 13:30  env.env
-rw------- 1 root root    1391 Jun 14 09:34  install_chromium_driver.sh
drwx------ 2 root root    4096 May 25 09:07  .ipynb_checkpoints/
-rw------- 1 root root    5022 Jun  9 16:44  keyword_categories.json
-rw------- 1 root root   11189 Jun 15 06:56  keyword_categories_v1.json
drwx------ 2 root root    4096 May 25 08:43  logs/
-rw------- 1 root root  179203 Jun  3 02:13  ner.ipynb
-rw------- 1 root root  405969 Jun 15 06:28 'recommender final.ipynb'
-rw------- 1 root root  425907 Jun 14 16:02 'recommender final_v1.ipynb'
-rw------- 1 root root   24496 Jun 14 09:0

In [ ]:
!pip install -Uq google-colab pandas supabase python-dotenv tensorflow_recommenders tensorflow-datasets

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.3/12.3 MB 83.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 96.2/96.2 kB 10.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 71.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 48.3/48.3 kB 4.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 71.5/71.5 kB 6.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 55.1/55.1 kB 5.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 184.3/184.3 kB 18.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 216.4/216.4 kB 18.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 135.9/135.9 kB 12.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 69.6/69.6 kB 7.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 106.8/106.8 kB 9.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.7/62.7 kB 6.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [ ]:
from supabase import Client, create_client
from dotenv import load_dotenv
from os import environ

load_dotenv("env.env")
supabase: Client = create_client(
    supabase_url=environ.get("DB_URL"),
    supabase_key=environ.get("DB_KEY")
)

response = supabase.table("gmap_id_2").select("*").execute()
data = response.data


df = pd.DataFrame(data)

In [ ]:
hotel_df = df[df.keyword_category.apply(lambda x: x is not None and 'penginapan' in x)]
hotel_df.head()

,id,name,map_url,img_url,rating,total_review,pricing,address,city,category,about,opening_hours,keyword_category,latitude,longitude
8,53057cbc-1e50-42ba-a060-6187701cc359,Penginapan Backpacker Bantul,https://www.google.com/maps/place/Penginapan+B...,https://streetviewpixels-pa.googleapis.com/v1/...,5.0,3.0,None,"48FG+9Q7, Jl. Kyai Ageng Teram, Teruman, Bantu...",Kabupaten Bantul,{},{},{},['penginapan'],-7.87660,110.327
10,0c0fc632-a1dd-4e87-a538-4bb980092c0c,Pari Solata Seaview Homestay,https://www.google.com/maps/place/Pari+Solata+...,https://lh5.googleusercontent.com/p/AF1QipPO4s...,4.2,256.0,None,"Jalan Dermaga No.2, Pulau Pari, Kec. Kepulauan...",Kabupaten Kepulauan Seribu,{},{},{},['penginapan'],-5.85846,106.620
13,7f130e13-978c-42f8-87d9-1540b4e0e077,Pelangi Rooms @ City Park Apartment,https://www.google.com/maps/place/Pelangi+Room...,https://lh3.googleusercontent.com/gps-proxy/AE...,4.5,122.0,None,"Ruko Citypark No.7, RW.14, Cengkareng Tim., Ke...",Kota Jakarta Barat,"{""pelayanan"": ""19"", ""strategis"": ""6"", ""karyawa...",{},{},['penginapan'],-6.13149,106.735
17,b352a766-c496-41a1-8468-5a974807b7e3,Penginapan & Rumah Makan ROOSYTA,https://www.google.com/maps/place/Penginapan+%...,https://lh5.googleusercontent.com/p/AF1QipPpey...,4.6,20.0,None,"Jl. Wediombo Jungwok, Nglaban, Jepitu, Kec. Gi...",Kabupaten Gunung Kidul,{},{},{},['penginapan'],-8.18284,110.710
21,a1b47154-d5d9-4842-b063-afa6e3518020,Penginapan Mutiara Laut,https://www.google.com/maps/place/Penginapan+M...,https://lh5.googleusercontent.com/p/AF1QipOj6X...,4.4,81.0,None,"Pulau Pari RT 001 RW 04, Pulau Pari, Kec. Kepu...",Kabupaten Kepulauan Seribu,{},{},{},['penginapan'],-5.85718,106.622


In [ ]:
hotel_df.shape

(499, 15)

In [ ]:
filtered_hotel_df = hotel_df[hotel_df['rating'] >= 4.4]
filtered_hotel_df = filtered_hotel_df[filtered_hotel_df['total_review'] > 30]
filtered_hotel_df.head()

,id,name,map_url,img_url,rating,total_review,pricing,address,city,category,about,opening_hours,keyword_category,latitude,longitude
13,7f130e13-978c-42f8-87d9-1540b4e0e077,Pelangi Rooms @ City Park Apartment,https://www.google.com/maps/place/Pelangi+Room...,https://lh3.googleusercontent.com/gps-proxy/AE...,4.5,122.0,None,"Ruko Citypark No.7, RW.14, Cengkareng Tim., Ke...",Kota Jakarta Barat,"{""pelayanan"": ""19"", ""strategis"": ""6"", ""karyawa...",{},{},['penginapan'],-6.13149,106.735
21,a1b47154-d5d9-4842-b063-afa6e3518020,Penginapan Mutiara Laut,https://www.google.com/maps/place/Penginapan+M...,https://lh5.googleusercontent.com/p/AF1QipOj6X...,4.4,81.0,None,"Pulau Pari RT 001 RW 04, Pulau Pari, Kec. Kepu...",Kabupaten Kepulauan Seribu,{},{},{},['penginapan'],-5.85718,106.622
26,c816470e-17eb-4c92-8dfc-ea3c318c2eea,Penginapan zhadela,https://www.google.com/maps/place/Penginapan+z...,https://lh3.googleusercontent.com/gps-proxy/AE...,5.0,45.0,None,"Jl. Pantai Utara No.5, RT.5/RW.3, Pulau Tidung...",Kabupaten Kepulauan Seribu,{},{},{},['penginapan'],-5.79534,106.496
40,607af3ee-33e3-427b-9e6f-6d9c21ecdcca,Acala Ayu,https://www.google.com/maps/place/Acala+Ayu/da...,https://lh5.googleusercontent.com/p/AF1QipMNEr...,4.7,392.0,None,"Jl. Ke Puncak Tim., Rejosari, Kemadang, Kec. T...",Kabupaten Gunung Kidul,"{""pantai"": ""45"", ""pemandangan"": ""25"", ""menu"": ...",{},{},['penginapan'],-8.13284,110.552
62,ac149126-db3e-4fb9-8c37-52a7181232d6,RedDoorz near Taman Palem,https://www.google.com/maps/place/RedDoorz+nea...,https://lh5.googleusercontent.com/p/AF1QipMKPi...,4.4,444.0,None,"Jl. Perumahan K. F. T, Cengkareng Bar., Kecama...",Kota Jakarta Barat,{},{},{},['penginapan'],-6.13569,106.725


In [ ]:
filtered_hotel_df[filtered_hotel_df.map_url.isna()]

,id,name,map_url,img_url,rating,total_review,pricing,address,city,category,about,opening_hours,keyword_category,latitude,longitude


In [ ]:
print('Hotel data in each city:')
print(filtered_hotel_df.city.value_counts())

Restaurant data in each city:
Kabupaten Bantul              53
Kabupaten Gunung Kidul        31
Kota Jakarta Pusat            19
Kabupaten Sleman              14
Kota Yogyakarta               12
Kota Jakarta Utara            12
Kabupaten Kepulauan Seribu    11
Kota Jakarta Selatan           8
Kota Jakarta Barat             7
Kabupaten Kulon Progo          6
Kota Jakarta Timur             4
Name: city, dtype: int64


In [ ]:
def hotel_recommender(city, top_k=3):
    hotel_by_city_df = filtered_hotel_df[filtered_hotel_df['city'] == city]
    random_rows = hotel_by_city_df.sample(n=3)
    return random_rows

In [ ]:
res = hotel_recommender('Kota Jakarta Barat')

In [ ]:
json_data = res.to_json(orient='records')
json_data

'[{"id":"fbb833ed-8ead-407d-bed0-5df528d58d7f","name":"Aston Kartika Grogol Hotel & Pusat Konferensi","map_url":"https:\\/\\/www.google.com\\/maps\\/place\\/Aston+Kartika+Grogol+Hotel+%26+Pusat+Konferensi\\/data=!4m10!3m9!1s0x2e69f70cc6ada809:0x1d0e960b29488705!5m2!4m1!1i2!8m2!3d-6.1675366!4d106.7921424!16s%2Fg%2F11gsbq3xv2!19sChIJCaitxgz3aS4RBYdIKQuWDh0?authuser=0&hl=id&rclk=1","img_url":"https:\\/\\/lh5.googleusercontent.com\\/p\\/AF1QipPcbDCFZYHkbbbpu_lJ2D2fDZlcwWzCUPVxTFD3=w408-h612-k-no","rating":4.8,"total_review":5102.0,"pricing":null,"address":"Kartika Tower, Jl. Kyai Tapa No.101, RT.4\\/RW.16, Tomang, Kec. Grogol petamburan, Kota Jakarta Barat, Daerah Khusus Ibukota Jakarta 11440, Indonesia","city":"Kota Jakarta Barat","category":"{}","about":"{}","opening_hours":"{}","keyword_category":"[\\"penginapan\\", \\"kegiatan\\"]","latitude":-6.16754,"longitude":106.792},{"id":"ac149126-db3e-4fb9-8c37-52a7181232d6","name":"RedDoorz near Taman Palem","map_url":"https:\\/\\/www.google.c